<font size="6"><b>IMPORT/EXPORT DATA FROM/TO EXCEL FILES</b></font>

In [ ]:
library(data.table)
library(tidyverse)
library(xlsx)
library(readxl)
library(tidyxl)

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=10) # for limiting the number of top and bottom rows of tables printed 

![xkcd](../imagesbb/spreadsheets.png)

(https://xkcd.com/2180)

Here we will show three alternative packages two write and/or read data into/from R and excel

Let's create a data.table copy of built-in iris

In [ ]:
iris2 <- iris

In [ ]:
setDT(iris2)

In [ ]:
iris2 %>% str

In [ ]:
iris2 %>% head

# xlsx package

Write iris2 into an excel file:

In [ ]:
write.xlsx(iris2, row.names = F, "~/databb/temp/iris.xlsx")

Read from excel into a data.frame and convert to data.table:

In [ ]:
iris_from_xl1 <- read.xlsx("~/databb/temp/iris.xlsx", sheetIndex = 1)

In [ ]:
setDT(iris_from_xl1)

Change Species from character to factor:

In [ ]:
iris_from_xl1[, Species := factor(Species, levels(iris$Species))]

The first table imported from excel is also identical with the original one:

In [ ]:
identical(iris2, iris_from_xl1)

# readxl package

Similar to xlsx package:

In [ ]:
iris_from_xl2 <- read_xlsx("~/databb/temp/iris.xlsx", sheet = 1)

In [ ]:
setDT(iris_from_xl2)

In [ ]:
iris_from_xl2[, Species := factor(Species, levels(iris$Species))]

The second table imported from excel is also identical with the original one:

In [ ]:
identical(iris2, iris_from_xl2)

# tidyxl

tidyxl package is an overkill for excel files with a regular format

But if the excel sheets have multiple tables of complex structure (multiple header column/rows, formatting relavant for data, formulas to extract, etc), tidyxl is a powerhouse: It read data cellwise into a long object with all metadata, format and coordinates as separate columns:

In [ ]:
iris_from_xl_long <- tidyxl::xlsx_cells("~/databb/temp/iris.xlsx", sheet = 1)

In [ ]:
setDT(iris_from_xl_long)

See how large a number of metadata is imported:

In [ ]:
iris_from_xl_long %>% str

Select relevant columns:

In [ ]:
iris_from_xl_long_2 <- iris_from_xl_long %>% select(row, col, is_blank, data_type, character, numeric)

And filter for only the numeric values with coordinate information (rows and columns)

In [ ]:
values1 <- iris_from_xl_long_2[data_type == "numeric", .(row, col, values = numeric)]

In [ ]:
values1

Now let's filter for character values. These include column headers and the Species column:

In [ ]:
iris_from_xl_long_3 <- iris_from_xl_long_2[data_type == "character"]

Column names reside in the first row:

In [ ]:
colnamesx <- iris_from_xl_long_3[row == 1, .(col, colname = character)]

In [ ]:
colnamesx

Species values reside on the rest of the rows:

In [ ]:
values2 <- iris_from_xl_long_3[row != 1, .(row, col, values = character)]

In [ ]:
values2

Now we have two data.tables for values: One for numeric columns, the other for species. Both are in long format now

Now join the column names through `col`:

In [ ]:
values1b <- values1 %>% left_join(colnamesx, by = "col")

In [ ]:
values2b <- values2 %>% left_join(colnamesx, by = "col")

And convert into wide format:

In [ ]:
values1c <- values1b %>% dcast(row ~ colname, value.var = "values")

In [ ]:
values1c

In [ ]:
values2c <- values2b %>% dcast(row ~ colname, value.var = "values")

In [ ]:
values2c

Join both tables using `row`:

In [ ]:
iris_from_xl3 <- values1c %>% left_join(values2c, by = "row") %>% select(-row)

Change Species into a factor:

In [ ]:
iris_from_xl3[, Species := factor(Species, levels(iris$Species))]

Reorder columns according to iris2:

In [ ]:
setcolorder(iris_from_xl3, names(iris2))

The `sorted` attribute is added during the `dcast` operation.

That's the only difference with iris2, let's delete that attribute

In [ ]:
attributes(iris_from_xl3)$sorted <- NULL

The third table imported from excel is also identical with the original one:

In [ ]:
identical(iris2, iris_from_xl3)